In [93]:
import pandas as pd
import numpy as np

In [94]:
ttc = pd.read_csv('ttc_delays_cleaned.csv')

In [95]:
ttc_stations = ttc['Station']
ttc_stations.head(2)

0        HIGH PARK
1    SHEPPARD - YU
Name: Station, dtype: object

In [96]:
stations = pd.read_csv('csv_originals/station_ridership_grade.csv')
stations['Station'] = stations['Station'].str.upper()
stations.head(2)

,Station,Grade,2015 ridership
0,COLLEGE,Underground,"47,790"
1,DAVISVILLE,Surface,"25,330"


In [97]:
station_index = stations.set_index('Station')
station_index.index

Index(['COLLEGE', 'DAVISVILLE', 'DUNDAS', 'DUPONT', 'EGLINTON',
       'EGLINTON WEST', 'FINCH', 'GLENCAIRN', 'KING', 'LAWRENCE',
       'LAWRENCE WEST', 'MUSEUM', 'NORTH YORK CENTRE', 'OSGOODE', 'QUEEN',
       'QUEEN'S PARK', 'ROSEDALE', 'SHEPPARD WEST', 'ST. ANDREW', 'ST. CLAIR',
       'ST. CLAIR WEST', 'ST. PATRICK', 'SUMMERHILL', 'UNION', 'WELLESLEY',
       'WILSON', 'YORK MILLS', 'YORKDALE', 'BLOOR-YONGE - BD',
       'BLOOR-YONGE - YU', 'SPADINA - YU', 'SPADINA - BD', 'ST. GEORGE - YU',
       'ST. GEORGE - BD', 'SHEPPARD - SHP', 'SHEPPARD - YU', 'BATHURST', 'BAY',
       'BROADVIEW', 'CASTLE FRANK', 'CHESTER', 'CHRISTIE', 'COXWELL',
       'DONLANDS', 'DUFFERIN', 'DUNDAS WEST', 'GREENWOOD', 'HIGH PARK',
       'ISLINGTON', 'JANE', 'KEELE', 'KIPLING', 'LANSDOWNE', 'MAIN STREET',
       'OLD MILL', 'OSSINGTON', 'PAPE', 'ROYAL YORK', 'RUNNYMEDE',
       'SHERBOURNE', 'VICTORIA PARK', 'WARDEN', 'WOODBINE', 'KENNEDY - SRT',
       'KENNEDY - BD', 'ELLESMERE', 'LAWRENCE EAST', 'MCC

In [98]:
ttc_stations_index = pd.Index(ttc_stations.unique())
ttc_stations_index

Index([         'HIGH PARK',      'SHEPPARD - YU',          'LANSDOWNE',
         'BLOOR-YONGE - YU',           'DUFFERIN',  'NORTH YORK CENTRE',
                'RUNNYMEDE',              'QUEEN',         'ST. ANDREW',
                'WELLESLEY',            'KIPLING',        'ST. PATRICK',
             'KENNEDY - BD',               'JANE',           'OLD MILL',
                   'WILSON',          'DOWNSVIEW',                'BAY',
                    'FINCH',            'COXWELL',     'ST. CLAIR WEST',
                   'DUPONT',         'YORK MILLS',        'DUNDAS WEST',
                  'OSGOODE',            'COLLEGE',   'BLOOR-YONGE - BD',
            'VICTORIA PARK',    'ST. GEORGE - YU',        'YONGE - SHP',
                'BROADVIEW',           'EGLINTON',           'ROSEDALE',
                     'PAPE',          'GREENWOOD',          'ISLINGTON',
                   'WARDEN',         'DAVISVILLE',           'LAWRENCE',
            'EGLINTON WEST',              'BLOOR', 

Lets find which of the Stations don't match

In [99]:
station_index.index.difference(ttc_stations_index)

Index(['MAIN STREET', 'SHEPPARD WEST'], dtype='object')

In [100]:
ttc_stations_index.difference(station_index.index)

Index([          nan,       'BLOOR',   'DOWNSVIEW',     'KENNEDY',
              'MAIN',    'SHEPPARD',     'SPADINA',  'ST. GEORGE',
             'YONGE', 'YONGE - SHP'],
      dtype='object')

In [ ]:
Looks like we need to fix:
BLOOR–YONGE --> Bloor and Yonge?
MAIN STREET --> MAIN
SHEPPARD WEST --> SHEPPARD
SHEPPARD–YONGE --> DOWNSVIE

In [101]:
# Remove the STREET from main street like we did in the other file
stations.loc[stations['Station'] == 'MAIN STREET'] = 'MAIN'

# Rename Sheppard West to DOWNSVIEW because this data predates 2017
stations.loc[stations['Station'] == 'SHEPPARD WEST'] = 'DOWNSVIEW'

In [102]:
station_index = stations.set_index('Station')
ttc_stations_index = pd.Index(ttc_stations.unique())
station_index.index.difference(ttc_stations_index)

Index([], dtype='object')

In [103]:
ttc_stations_index.difference(station_index.index)

Index([          nan,       'BLOOR',     'KENNEDY',    'SHEPPARD',
           'SPADINA',  'ST. GEORGE',       'YONGE', 'YONGE - SHP'],
      dtype='object')

In [61]:
ttc_exchange_stations = ttc[ttc['Station'].isin(['BLOOR', 'SHEPPARD', 'YONGE'])]

In [73]:
ttc_exchange_stations[['Station', 'Line']].drop_duplicates().sort_values('Station')

,Station,Line
3,BLOOR,Yonge-University
69,BLOOR,NaN
142,BLOOR,Bloor-Danforth
1,SHEPPARD,Yonge-University
1413,SHEPPARD,Sheppard
15880,SHEPPARD,NaN
20421,SHEPPARD,Bloor-Danforth
38,YONGE,Bloor-Danforth
42,YONGE,Sheppard
73,YONGE,Yonge-University


In [75]:
ttc_exchange_stations.groupby(['Station', 'Line']).size()

Station   Line            
BLOOR     Bloor-Danforth      1835
          Yonge-University    1634
SHEPPARD  Bloor-Danforth        37
          Sheppard              50
          Yonge-University    1094
YONGE     Bloor-Danforth      1703
          Sheppard            1031
          Yonge-University    2802
dtype: int64